In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import make_column_transformer
import matplotlib.pyplot as plt
import seaborn as sns
from fancyimpute import IterativeImputer

In [ ]:
#Lets start-off by loading data
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")


### Getting Started

For future processings, I will concatinate two sets together. So that training set and test set will have the same number of features in the same format. This will provide great ease to process when the models are being implemented. 

In order to preserve the boarderline between training set and test set, I am recording the initial number of rows each has. Although the number of columns and the data wihtin those columns will change in future, number of rows will remain in tact. 

In [ ]:
n_train = train.shape[0]
n_test = test.shape[0]

test_id = test["Id"]

train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)


# Exploratory Data Analysis
   ## 1. Missing Data

As a first step, I would like to get a look at the data, explore the data types in each columns and missing values. 

Showing the ratio between missing values and total values in given column will help me get an understanding on the severity of missing values. 
    

In [ ]:
print("AllData")
(all_data_rows, all_data_columns) = all_data.shape
print(" Number of rows: {} \n Number of columns: {}".format(all_data_rows, all_data_columns))
print(train.sample(3))

def display_missing(df):
    for col in df.columns.tolist():
        print("{} column missing values: {} / {}".format(col, df[col].isnull().sum(),df.shape[0]))

display_missing(all_data)

Above analysis provided info about missing values, now we can make an intiution on what is our threshold going to be for missing values. 

The columns with missing values surpassing the threshold will be dropped.  

From intuition, 20% is a good threshold for now. 

In [ ]:
def filterProblematicColumns(df,threshold):
    listOfColumnNames = []
    for col in df.columns.tolist():
        if df[col].isnull().sum()> threshold:
            listOfColumnNames.append(col)
            print(col)
    
    return listOfColumnNames

portion = 0.2
threshold = all_data.shape[0] * portion


columnsToDrop = filterProblematicColumns(all_data, threshold)

all_data = all_data.drop(columns=columnsToDrop)

Now that problematic values dropped, we can work on filling missing values that lie below the threshold.

We can start our work by analyzing the remaining columns with missing values. 

In [ ]:
columns_with_missing_values = all_data.loc[:, all_data.isnull().any()]
missing_columns = columns_with_missing_values.columns.tolist()

print("Columns with Missing Values: ","\n", "\n", missing_columns, "\n")
print(columns_with_missing_values.describe())
print(all_data.shape)
print("\n", "--------------", "\n")


Checking for correlation with a heatmap is a good idea to visualize relationships. Unfortunately, here there are two drawbacks. 

    1. There are too many features to get a sense of just by looking at correlation values or at a heatmap
    
    2. There are non-numeric columns within the dataset. Correlations only work between numeric data. 

In [ ]:
numcols = all_data.select_dtypes(include = np.number).columns

#Lets start by plotting a heatmap to determine if any variables are correlated
plt.figure(figsize = (12,8))
sns.heatmap(data= all_data[numcols].corr())
plt.show()
plt.gcf().clear()


In [ ]:
def corr_missing_values(df, columns): 
    for column in columns:
        df_corr = df.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
        df_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
        print(df_corr[df_corr['Feature 1'] == column])
        print("")

corr_missing_values(all_data, [x for x in missing_columns if x in numcols])


## 2.Encoding values

There are two reasons for encoding non-numeric values: 

    1- We have decided to deal with missing values using MICE. MICE can only fill missing values if other values are numeric. The regression can only be implemented for imputation, if the values are numbers. 
    
    2- The predictive models will be requiring numeric values for the modeling process. 
    

There are different consideration for encoding values. Ordinal values are values given in some sort of order in relation with eachother. Nominal values are values where such ordering does not exist. 

For ordinal values, I will be using OrdinalEncoder and for nominal values, I will be using dummies to encode. 

   ### 2a. Ordinal Values

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

numeric_columns = all_data.select_dtypes(include = np.number).columns.tolist()
nominal_columns = ["MSZoning","Street","LandContour","LotConfig","Neighborhood","Condition1","Condition2","BldgType","HouseStyle","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","Foundation","Heating","CentralAir","Electrical","GarageType","SaleCondition"]
ordinal_columns = ["LotShape","Utilities","LandSlope","ExterQual","ExterCond","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","HeatingQC","KitchenQual","Functional","GarageFinish","GarageQual","GarageCond","PavedDrive","SaleType"]

#Check if numbers match, to make sure no columns are left out
print(all_data.shape[1])
print(len(numeric_columns), len(nominal_columns), len(ordinal_columns))


While encoding, it is vital that we skip missing values. Because ordinal encoder does not work with missing values. We will impute those missing values, once everydata is turned into numeric. Then MICE will handle the missing values. 

In [ ]:
## Ordinal Encoding (by skipping null values)

ordinal_enc_dict = {}
for col_name in ordinal_columns:
    ordinal_enc_dict[col_name] = OrdinalEncoder()

    col = all_data[col_name]
    col_not_null = col[col.notnull()]
    reshaped_vals = col_not_null.values.reshape(-1,1)

    encoded_vals = ordinal_enc_dict[col_name].fit_transform(reshaped_vals)
    all_data.loc[col.notnull(), col_name] = np.squeeze(encoded_vals)

#Check if the values are encoded and no column has been skipped.   
print(all_data[ordinal_columns].head())

###   2b. Nominal Values

I have find hard time to find solution to encode nominal values with missing data. I am open to suggestions here. Please comment below if you have any advice. 

Since dummies do not work with missing values. I choosed a more standard approach to continue. I have investigated nominal columns with missing data. I have discovered the amount of missing data is relatively low. I have choosed to fill those missing values on the basis of occurence. 

In [ ]:
print(display_missing(all_data[nominal_columns]))

In [ ]:
## Imputation with mode
nom_cols_withnull = all_data[nominal_columns].columns[all_data[nominal_columns].isnull().any()].tolist()

most_common_imputed = all_data[nom_cols_withnull].apply(lambda x: x.fillna(x.value_counts().index[0]))

for col_name in most_common_imputed.columns:
    all_data[col_name] = most_common_imputed[col_name]

#### Encoding nominal values using dummies

In [ ]:
nom_df = pd.get_dummies(all_data[nominal_columns], prefix=nominal_columns)

for col_name in nom_df.columns:
    all_data[col_name] = nom_df[col_name]

all_data = all_data.drop(columns= nominal_columns)

print(all_data)

## 3. Imputation with MICE

In [ ]:
from fancyimpute import IterativeImputer


MICE_imputer = IterativeImputer()
ordinal_mice = all_data.copy(deep = True)

ordinal_mice.iloc[:,:] = np.round(MICE_imputer.fit_transform(ordinal_mice))

for col_name in ordinal_columns:
    all_data[col_name] = ordinal_mice[col_name]

for col_name in numeric_columns:
    all_data[col_name] = ordinal_mice[col_name]



In [ ]:
if all_data.isnull().values.any():
    print("Yuh artık!")
    print("GOSHHH!!!!!")
    print("Breakdown loading...")

## 4. Splitting the Concatinated DataSet and Feature Scaling

After all the processes, we can divide the training and test set from each other for model testing. 

Feature Scaling is a crutial step here to regularize the weight of each parameter to the outcome. Otherwise, values with greater magnitude will overweight in the model. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()


y = train["SalePrice"]
X = all_data.loc[:n_train-1,:]
test = all_data.loc[n_train:,:]


X.iloc[:,:] = scaler.fit_transform(X.loc[:,:])
test.iloc[:,:] = scaler.fit_transform(test.loc[:,:])

X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.20, random_state=1)


## 5. Hyperparameter tuning and GridSearchCV

In hyperparameter tuning, the main purpose is to find parameters that give out best predictions for validation set. This way, we avoid overfitting and get more generalized result for real tests. 

GridSearchCV is a decent tool to try different parameters while crossvalidating trials on the test set


In [ ]:
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import metrics
from sklearn.metrics import mean_squared_error



# create and fit a ridge regression model, testing each alpha
ridge = Ridge()
lasso = Lasso()
lr = LinearRegression()

kernel_ridge = KernelRidge()

param_grid_kr = {'alpha': [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
              'kernel':['polynomial'], 
              'degree':[2,3,4,5,6,7,8],
              'coef0':[0,1,1.5,2,2.5,3,3.5,10]}
### 
#kernel_ridge = GridSearchCV(kernel_ridge, 
 #                param_grid = param_grid_kr, 
  #               scoring = "neg_mean_squared_error", 
   #             cv = 5,
    #             n_jobs = -1,
     #            verbose = 1)

#kernel_ridge.fit(X,y)
#print(pd.DataFrame(kernel_ridge.cv_results_))
#k_best = kernel_ridge.best_estimator_
#kernel_ridge.best_score_

param_grid = {"alpha": [0.001,0.003,0.01,0.3,0.1,0.3,1,3,10,30,100,300,1000,3000,100000]}

print("-----------Stats for Ridge-----------------", "\n")
grid_ridge = GridSearchCV(ridge, param_grid = param_grid, cv = 10, scoring = "neg_mean_squared_error", n_jobs=-1) 
grid_ridge.fit(X, y)
print(pd.DataFrame(grid_ridge.cv_results_))

print("-----------Stats for Lasso-----------------", "\n")
grid_lasso = GridSearchCV(lasso, param_grid = param_grid, cv = 10, scoring = "neg_mean_squared_error", n_jobs=-1) 
grid_lasso.fit(X_train, y_train)
print(pd.DataFrame(grid_lasso.cv_results_))

#print(kernel_ridge.best_score_)
#print(kernel_ridge.best_estimator_.alpha)

print("-----------Scoreboard for Ridge-----------------", "\n")
print(grid_ridge.best_score_)
print(grid_ridge.best_estimator_.alpha)

print("-----------Scoreboard for Lasso-----------------", "\n")

print(grid_lasso.best_score_)
print(grid_lasso.best_estimator_.alpha)

### Submission

In [ ]:
test_ridge = Ridge(alpha = 300)
test_ridge.fit(X,y)
predictions = test_ridge.predict(test)


#output = pd.DataFrame({'Id': test_id, 'SalePrice': predictions})
#output.to_csv('first_draft.csv', index=False)



Please feel free to add comments and give recommendations on the work. I would be happy to improve myself. I hope this will help you! 